# Delivery time prediction

Justyna Gibas 403122 <br>
Jan Pomykacz

## Introduction
The goal of our project is to predict food delivery time. Model can be used by companies to estimate delivery time based on distance, order time etc, and notify their customers when food should arrive. Such notification are implemented in services (e.g Pyszne.pl), but often they're incorrect.

## Dataset
In project we use data from Kaggle only from file train.csv (https://www.kaggle.com/datasets/gauravmalik26/food-delivery-dataset?select=train.csv).  Both numerical and categorical variable are available. Data comes from orders in India. Dataset contain information about:
-	order ID and delivery person ID,
-	delivery person age and rating,
-	coordinates of restaurant,
-	coordinates of delivery location,
-	order date and time,
-	pickup time (by delivery person),
-	weather conditions,
-	traffic,
-	vehicle type and it’s condition,
-	type of food ordered,
-	city type, 
-	information if there is a festival,
-	number of deliveries,
-	number of simultaneously delivered orders,
-	delivery time (in minutes). <br><br>

We don’t take into consideration IDs, information about festivals (are very rare 1-2% of data) and vehicle condition (values aren’t described)



## DAG

## Possible cofoundings

## Data Preprocessing

To prepare data for modelling and better them understand we take the following  steps:
1.	delete wrong coordinates – Some records contain coordinates that don’t belong to India e.g. ocean point. We remove them due to we can't create road paths between this points.
2.	remove records containing nulls – We don’t consider missing or corrupted predictors/data, because they make prediction impossible.
3.	calculate meal preparation time – There was only date and time for of order and pick up by delivery person. We calculate time in minutes to simplified modelling.
4.	delete impossible delivery times – Some records contain delivery time that is lower or equal to meal preparation time. We assumed that meal preparation time is part of delivery time so those records would have negative travel time.
5.	converting coordinates to road distance – We mapped the coordinates to a distance to better match the actual delivery route.  We used OpenStreetMaps API.  Some created routes have incorrect distances or failed to create.
6.	removing corrupted routes – There were some issue with routing – road distances was lower that straight line distance, returned distance was abnormal (90 km) ???. 
7.	filtering data – We assumed that maximum delivery distance is around 20 km.  In modelling we used records that have distances lower or equal to 20.
8.	mapping categorical (string) data to numerical – In our models we used categorical value about traffic level, but models cannot use non-numerical predictors. 

Note that only filtering and mapping are done in this notebook.  You can find data after preprocessing (steps 1-6) in file delivery_time_data.csv. 


# Models description

We designed two models  to make a delivery time prediction. Both models using inverse gamma distribution to model the delivery time. In second model we additionally take into account parameters describing delivery person – his/her rating and number of deliveries.



## Model 1

$$ delivery\_time_{i} \sim InvGamma(\alpha_{i}, \beta_{i})$$
$$ \alpha_i=\frac{\mu_i^2}{\sigma_i^2}+2$$
$$ \beta=\frac{\mu_i^3}{\sigma_i^2}+\mu_i $$
$$ \sigma_i \sim Exponential\left(0.5\right)$$
$$ \mu_i = exp(distance\_coeff_{i} \cdot distance_{i} +  traffic\_level\_coeff[traffic\_level_{i}]\ + meal\_prep\_coeff_{i}\ \cdot  meal\_prep\_time_{i}) $$ 
$$ distance\_coeff_{i}  \sim Normal(0,0.3)$$ 
$$ meal\_prep\_coeff_{i} \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff_{i} \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[1] \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[2] \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[3] \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[4] \sim Normal(0,0.3)$$

## Model 2


$$ delivery\_time_{i} \sim InvGamma(\alpha_{i}, \beta_{i})$$
$$ \alpha_i=\frac{\mu_i^2}{\sigma_i^2}+2$$
$$ \beta=\frac{\mu_i^3}{\sigma_i^2}+\mu_i $$
$$ \sigma_i \sim Exponential\left(0.5\right)$$
$$ \mu_i = exp(distance\_coeff_{i} \cdot distance_{i}+traffic\_level\_coeff[traffic\_level_{i}]\ + meal\_prep\_coeff_{i}\ \cdot  meal\_prep\_time_{i} + \\ + person\_raiting\_coeff_{i} \cdot person\_raiting_{i} +  deliveries\_number\_coeff[number\_of\_deliveiers_{i}])$$ 
$$ distance\_coeff_{i}  \sim Normal(0,0.3)$$ 
$$ meal\_prep\_coeff_{i} \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff_{i} \sim Normal(0,0.3)$$
$$ person\_raiting\_coeff_{i} \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[1] \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[2] \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[3] \sim Normal(0,0.3)$$
$$ traffic\_level\_coeff[4] \sim Normal(0,0.3)$$
$$ deliveries\_number\_coeff[1] \sim Normal(0,0.3)$$
$$ deliveries\_number\_coeff[2] \sim Normal(0,0.3)$$
$$ deliveries\_number\_coeff[3] \sim Normal(0,0.3)$$
$$ deliveries\_number\_coeff[4] \sim Normal(0,0.3)$$